In [23]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.base import BaseEstimator, TransformerMixin

import spacy
import re

In [24]:
nlp = spacy.load("en_core_web_sm")

In [25]:
# load the dataset
data = pd.read_csv('historical_task_data.csv')

In [26]:
# custom transformer for feature extraction
class TextFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        num_tokens = []
        max_token_length = []
        min_token_length = []
        avg_token_length = []
        num_stopwords = []
        fraction_stopwords = []
        num_entities = []
        num_names = []
        has_phone_number = []
        num_action_words = []
        num_depth_words = []
        num_pos_tags = []

        # extract features from each text
        for text in X:
            doc = nlp(text)
            tokens = [token.text for token in doc]
            num_tokens.append(len(tokens))
            token_lengths = [len(token) for token in tokens]
            max_token_length.append(max(token_lengths))
            min_token_length.append(min(token_lengths))
            avg_token_length.append(np.mean(token_lengths))
            num_stopwords.append(sum(1 for token in doc if token.is_stop))
            fraction_stopwords.append(sum(1 for token in doc if token.is_stop) / len(tokens))
            num_entities.append(len(doc.ents))
            num_names.append(sum(1 for ent in doc.ents if ent.label_ == "PERSON"))
            has_phone_number.append(bool(re.search(r'\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}', text)))
            num_action_words.append(len([token.text.lower() for token in doc if token.pos_ == 'VERB']))
            num_depth_words.append(len([token.text.lower() for token in doc if token.text.lower() in ['deep', 'detailed', 'careful', 'thorough', 'overview', 'lightweight', 'light', 'end to end', 'e2e']]))
            num_pos_tags.append(len(set([token.pos_ for token in doc])))

        # convert lists to arrays
        num_tokens = np.array(num_tokens).reshape(-1, 1)
        max_token_length = np.array(max_token_length).reshape(-1, 1)
        min_token_length = np.array(min_token_length).reshape(-1, 1)
        avg_token_length = np.array(avg_token_length).reshape(-1, 1)
        num_stopwords = np.array(num_stopwords).reshape(-1, 1)
        fraction_stopwords = np.array(fraction_stopwords).reshape(-1, 1)
        num_entities = np.array(num_entities).reshape(-1, 1)
        num_names = np.array(num_names).reshape(-1, 1)
        has_phone_number = np.array(has_phone_number).reshape(-1, 1)
        num_action_words = np.array(num_action_words).reshape(-1, 1)
        num_depth_words = np.array(num_depth_words).reshape(-1, 1)
        num_pos_tags = np.array(num_pos_tags).reshape(-1, 1)

        # combine feature arrays
        features = np.concatenate((num_tokens, max_token_length, min_token_length, avg_token_length,
                                   num_stopwords, fraction_stopwords, num_entities, num_names,
                                   has_phone_number, num_action_words, num_depth_words, num_pos_tags), axis=1)

        return features

In [27]:
# custom transformer to concatenate text columns and apply TF-IDF vectorization and feature extraction
class ConcatenateTextColumns(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # combine 'task Name' and 'task Description' columns
        concatenated_text = X['Task Name'] + ' ' + X['Task Description']

        return concatenated_text

In [28]:
# apply feature extraction
concatenator = ConcatenateTextColumns()
X_all_text = concatenator.transform(data)

text_feature_extractor = TextFeatureExtractor()
X_all_features = text_feature_extractor.transform(X_all_text)

In [29]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X_all_features, data['Task Duration'], test_size=0.2, random_state=42)

In [30]:
# modeling pipeline
pipeline = Pipeline(steps=[('regressor', RandomForestRegressor())])

# training
pipeline.fit(X_train, y_train)

Pipeline(steps=[('regressor', RandomForestRegressor())])

In [31]:
# prediction
y_pred = pipeline.predict(X_test)

# evaluation
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 26.61916666666667


In [32]:
from joblib import dump, load

dump(pipeline, 'task_duration_model.joblib')

loaded_pipeline = load('task_duration_model.joblib')

new_task_name = input("Enter the new task name: ")
new_task_description = input("Enter the new task description: ")

# combine task name and description
new_task_text = new_task_name + ' ' + new_task_description

# transform input using the loaded model
new_task_features = text_feature_extractor.transform([new_task_text])

predicted_duration = loaded_pipeline.predict(new_task_features)

print(f"Predicted Duration for the New Task:{new_task_name}")
print(f"Predicted Duration for the New Task:{new_task_description}")
print(f"Predicted Duration for the New Task:{predicted_duration[0]} minutes")

Predicted Duration for the New Task:Finish software engineering final project
Predicted Duration for the New Task:Bug fix the app.py and index.html so that the task estimation prediction is displayed
Predicted Duration for the New Task:91.48 minutes


In [33]:
new_task_name = input("Enter the new task name: ")
new_task_description = input("Enter the new task description: ")

# combine task name and description
new_task_text = new_task_name + ' ' + new_task_description

# transform input using the loaded model
new_task_features = text_feature_extractor.transform([new_task_text])

predicted_duration = loaded_pipeline.predict(new_task_features)

print(f"Predicted Duration for the New Task:{new_task_name}")
print(f"Predicted Duration for the New Task:{new_task_description}")
print(f"Predicted Duration for the New Task:{predicted_duration[0]} minutes")

Predicted Duration for the New Task:Eat pizza.
Predicted Duration for the New Task:Eat like 20 pizzas or something.
Predicted Duration for the New Task:36.62 minutes
